In [1]:
from utils import get_train_test_datasets
import pandas as pd
from autogluon.tabular import TabularPredictor
import pandas as pd

/home/julian/miniconda3/envs/dtiam-esm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
included_model_types = ['NN_TORCH', 'FASTAI', 'GBM', 'XGB']
hyperparameters = {
	'NN_TORCH': [{'ag_args_fit': {'num_gpus': 1}}],
	'GBM': [
        {'ag_args_fit': {'num_gpus': 1}},
        {'learning_rate': 0.03, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 3, 'ag_args': {'ag_args_fit': {'num_gpus': 1}, 'name_suffix': 'Large', 'priority': 0, 'hyperparameter_tune_kwargs': None}}],
	'XGB': [{'ag_args_fit': {'num_gpus': 1}}],
	'FASTAI': [{'ag_args_fit': {'num_gpus': 1}}]
}
eval_metric = 'roc_auc'
preset = None
problem_type = 'binary'

In [3]:
# dataset = 'activation'
dataset = 'inhibition'
setting = 'warm_start'
# setting = 'drug_coldstart'
# setting = 'protein_coldstart'
target = 'moa'
fold = 0
mol_model = 'MolE_GuacaMol_27113.ckpt'
# mol_model = 'BerMolModel_base.pkl'
# mol_model = 'dtiam_kiba_compound_features'
train_df, test_df = get_train_test_datasets(fold=fold, setting=setting, dataset=dataset, mol_model=mol_model, target=target)

Index(['drug_id', 'protein_id', 'affinity'], dtype='object')


In [4]:
for fold in range(5):
    dataset_path = "../data/dta/" + dataset
    folds_path = dataset_path + f"/data_folds/{setting}/"

    train_df, test_df = get_train_test_datasets(fold=fold, setting=setting, dataset=dataset, mol_model=mol_model, target=target)
    
    model_path = f"/home/julian/DTIAM/code/AutogluonModels/{dataset}-{setting}-fold_{fold}-{mol_model.split('.')[0]}"
    print(model_path)

    predictor = TabularPredictor(path=model_path, label="y", eval_metric=eval_metric, problem_type=problem_type).fit(
        train_data=train_df, included_model_types=included_model_types, presets=preset, hyperparameters=hyperparameters
    )

Index(['drug_id', 'protein_id', 'affinity'], dtype='object')


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.10.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Apr 17 19:20:47 UTC 2
CPU Count:          20
Memory Avail:       26.72 GB / 62.63 GB (42.7%)
Disk Space Avail:   192.88 GB / 1684.96 GB (11.4%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='high'         : Strong accuracy with 

/home/julian/DTIAM/code/AutogluonModels/inhibition-warm_start-fold_0-MolE_GuacaMol_27113


Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    27668.28 MB
	Train Data (Original)  Memory Usage: 1447.53 MB (5.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 2048 | ['0', '1', '2', '3', '4', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 2048 | ['0', '1', '2', '3', '4', ...]
	17.2s = Fit runtime
	2048 features in o

[1000]	valid_set's binary_logloss: 0.122963
[2000]	valid_set's binary_logloss: 0.108273
[3000]	valid_set's binary_logloss: 0.105907
[4000]	valid_set's binary_logloss: 0.107039


...

Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    17637.45 MB
	Train Data (Original)  Memory Usage: 1447.53 MB (8.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 2048 | ['0', '1', '2', '3', '4', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 2048 | ['0', '1', '2', '3', '4', ...]
	14.3s = Fit runtime
	2048 features in o

[1000]	valid_set's binary_logloss: 0.130524
[2000]	valid_set's binary_logloss: 0.114027
[3000]	valid_set's binary_logloss: 0.112611


	0.9671	 = Validation score   (roc_auc)
	1652.04s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	To avoid this warning, specify the model hyperparameter "ag.max_memory_usage_ratio" to a larger value (currently 1.0, set to >=1.20 to avoid the warning)
		To set the same value for all models, do the following when calling predictor.fit: `predictor.fit(..., ag_args_fit={"ag.max_memory_usage_ratio": VALUE})`
		Setting "ag.max_memory_usage_ratio" to values above 1 may result in out-of-memory errors. You may consider using a machine with more memory as a safer alternative.
	0.9677	 = Validation score   (roc_auc)
	241.36s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: XGBoost ...
	0.9711	 = Validation score   (roc_auc)
	74.48s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.9754	 = Validation score   (roc_auc)
	453.34s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: LightGBMLarg

[1000]	valid_set's binary_logloss: 0.113336


	0.9676	 = Validation score   (roc_auc)
	677.68s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetTorch': 0.632, 'NeuralNetFastAI': 0.263, 'XGBoost': 0.105}
	0.9781	 = Validation score   (roc_auc)
	0.03s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 3123.84s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 11896.5 rows/s (2500 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/home/julian/DTIAM/code/AutogluonModels/inhibition-warm_start-fold_4-MolE_GuacaMol_27113")


In [5]:
# ag-20250616_230514 es drug_coldstart

In [6]:
# predictor.path = mol_model

In [7]:
test_data_nolab = test_df.drop(columns=["y"])

In [8]:
pred_scores = predictor.predict(test_data_nolab)

In [9]:
import numpy as np
from utils import load_data, rmse, mse, pearson, spearman, ci, roc_auc, pr_auc
res_all = pd.DataFrame(columns=["RMSE", "MSE", "Pearson", "Spearman", "CI"])
G, P = np.array(test_df["y"]), np.array(pred_scores)
ret = [rmse(G, P), mse(G, P), pearson(G, P), spearman(G, P), ci(G, P)]

In [10]:
i = 0
print(
    f"fold: {i+1}, RMSE: {ret[0]}, MSE: {ret[1]}, Pearson: {ret[2]}, Spearman: {ret[3]}, CI: {ret[4]}"
)
res_all.loc[i] = ret

fold: 1, RMSE: 0.18102848733628069, MSE: 0.032771313227261935, Pearson: 0.798958836937087, Spearman: 0.798958836937095, CI: 0.891639104075121
